In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

# 2. Separa variables predictoras y objetivo
y = train['RENDIMIENTO_GLOBAL']
X = train.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
X_test = test.drop(['ID'], axis=1)

# 3. Cross-validation con barra de progreso y medición de tiempo
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
y_true = []
y_pred_cv = []

print("Iniciando validación cruzada...")
start_time = time.time()
for train_idx, val_idx in tqdm(cv.split(X, y), total=cv.get_n_splits(), desc="Validación cruzada"):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    score = clf.score(X_val, y_val)
    scores.append(score)
    # Para métricas detalladas:
    y_pred_fold = clf.predict(X_val)
    y_true.extend(y_val)
    y_pred_cv.extend(y_pred_fold)
elapsed = time.time() - start_time
print(f"Validación cruzada completada en {elapsed/60:.2f} minutos.")
print(f"CV Accuracy promedio: {np.mean(scores):.4f}")

# 4. Entrena el modelo final y predice sobre el test con barra de progreso
print("Entrenando modelo final con todos los datos...")
start_time = time.time()
with tqdm(total=1, desc="Entrenando modelo final") as pbar:
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X, y)
    pbar.update(1)
elapsed_final = time.time() - start_time
print(f"Entrenamiento final completado en {elapsed_final:.2f} segundos.")

y_pred = clf.predict(X_test)

# 5. Guarda el resultado para submission
submission = pd.DataFrame({'ID': test['ID'], 'RENDIMIENTO_GLOBAL': y_pred})
submission.to_csv('submission.csv', index=False)
print("Archivo 'submission.csv' generado.")

# 6. Resultados finales y explicación

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred_cv)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.title('Matriz de confusión (validación cruzada)')
plt.show()

# Reporte de clasificación
print("Reporte de clasificación (validación cruzada):")
print(classification_report(y_true, y_pred_cv))

# Importancia de variables
importances = clf.feature_importances_
features = X.columns
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10,6))
plt.title("Importancia de variables (Random Forest)")
plt.bar(range(len(features)), importances[indices])
plt.xticks(range(len(features)), features[indices], rotation=90)
plt.tight_layout()
plt.show()

# Explicación de los resultados:
print("""
Explicación de los resultados:
- La barra de progreso muestra el avance de la validación cruzada y el entrenamiento final.
- 'CV Accuracy promedio' es la precisión media obtenida en los 5 folds de validación cruzada.
- La matriz de confusión muestra cómo se distribuyen los aciertos y errores por clase.
- El reporte de clasificación incluye precisión, recall y F1-score para cada clase.
- El gráfico de importancia de variables te ayuda a identificar qué variables son más relevantes para el modelo.
- El archivo 'submission.csv' contiene las predicciones finales para el conjunto de test.
""")

In [17]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import multiprocessing as mp
import psutil  # Para monitorear RAM
import joblib
from tqdm import tqdm

def monitor_memory():
    """Monitorea el uso de memoria RAM"""
    memory = psutil.virtual_memory()
    used_gb = memory.used / (1024**3)
    total_gb = memory.total / (1024**3)
    available_gb = memory.available / (1024**3)
    
    print(f"MEMORIA RAM:")
    print(f"  Total: {total_gb:.1f} GB")
    print(f"  Usada: {used_gb:.1f} GB ({memory.percent:.1f}%)")
    print(f"  Disponible: {available_gb:.1f} GB")
    
    if memory.percent > 85:
        print("ADVERTENCIA: Uso de RAM alto (>85%)")
    elif memory.percent > 75:
        print("RAM en nivel óptimo para entrenamiento")
    else:
        print("RAM con suficiente espacio libre")
    
    return memory.percent

# Configuración optimizada para velocidad
print("CONFIGURACIÓN OPTIMIZADA PARA VELOCIDAD")
print("="*60)
print(f"Cores disponibles: {mp.cpu_count()}")
monitor_memory()
print("Configuración RÁPIDA - Menos iteraciones pero más eficiente")
print("="*60)

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

# 2. Separa variables predictoras y objetivo
y = train['RENDIMIENTO_GLOBAL']
X = train.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
X_test = test.drop(['ID'], axis=1)

print(f"Número de características: {X.shape[1]}")
print(f"Número de muestras de entrenamiento: {X.shape[0]}")
print(f"Número de muestras de test: {X_test.shape[0]}")

# 3. Pipeline con escalado y SVM optimizado para VELOCIDAD
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(
        probability=True, 
        random_state=42, 
        max_iter=2000,    # Reducido para velocidad
        cache_size=200,   # Reducido para evitar saturación
        shrinking=True,   # Optimización interna
        tol=1e-3         # Tolerancia más relajada para velocidad
    ))
])

# 4. Búsqueda de hiperparámetros OPTIMIZADA PARA VELOCIDAD
# Reducimos drásticamente el espacio de búsqueda
param_distributions = [
    # Solo RBF kernel - el más efectivo generalmente
    {
        'svc__C': [0.1, 1, 10, 100],  # Valores más focalizados
        'svc__kernel': ['rbf'],
        'svc__gamma': ['scale', 0.001, 0.01, 0.1],  # Menos valores
        'svc__class_weight': [None, 'balanced']
    },
    # Linear kernel para comparación rápida
    {
        'svc__C': [0.1, 1, 10, 100],
        'svc__kernel': ['linear'],
        'svc__class_weight': [None, 'balanced']
    }
]

# 5. Validación cruzada con menos pliegues para velocidad
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Reducido a 3 pliegues

cores_disponibles = mp.cpu_count()
cores_optimos = max(1, int(cores_disponibles * 0.8))  # Usamos más cores

print("\nBÚSQUEDA RÁPIDA DE HIPERPARÁMETROS")
print("="*60)
print("CONFIGURACIÓN OPTIMIZADA PARA VELOCIDAD")

# Estimación de tiempo optimizada
n_combinations = 32  # Estimación de combinaciones
total_fits = n_combinations * 3  # 3-fold CV
tiempo_estimado = total_fits * 0.5 / cores_optimos  # Estimación más agresiva

print(f"CONFIGURACIÓN RÁPIDA:")
print(f"   • Cores utilizados: {cores_optimos} de {cores_disponibles} disponibles")
print(f"   • Iteraciones de búsqueda: 20 (reducido)")
print(f"   • Validación cruzada: 3-fold (reducido)")
print(f"   • Total de entrenamientos: ~60 modelos SVM")
print(f"   • Tiempo estimado: {tiempo_estimado:.1f} minutos")

print(f"\nHora de inicio: {time.strftime('%H:%M:%S')}")
print("Modo RÁPIDO activado - Menos iteraciones, más velocidad")
print("="*60)

start_time = time.time()

# Configuración ultra-optimizada para velocidad
random_search = RandomizedSearchCV(
    pipe, 
    param_distributions=param_distributions, 
    n_iter=20,   # REDUCIDO DRÁSTICAMENTE para velocidad
    cv=cv,      # 3-fold en lugar de 5-fold
    scoring='accuracy', 
    n_jobs=cores_optimos,
    verbose=2,
    random_state=42,
    return_train_score=True,
    pre_dispatch='2*n_jobs'  # Más agresivo
)

# Wrapper con barra de progreso usando tqdm
def fit_with_progress(search_cv, X, y):
    """Entrena el modelo con barra de progreso"""
    n_iter = search_cv.n_iter
    
    print(f"Entrenando {n_iter} modelos SVM...")
    with tqdm(total=n_iter, desc='Progreso SVM', unit='modelo') as pbar:
        # Interceptar el método _run_search temporalmente
        original_run_search = search_cv._run_search
        
        def wrapped_run_search(evaluate_candidates):
            def wrapped_evaluate(candidate_params):
                results = evaluate_candidates(candidate_params)
                pbar.update(len(candidate_params))
                return results
            return original_run_search(wrapped_evaluate)
        
        search_cv._run_search = wrapped_run_search
        
        try:
            return search_cv.fit(X, y)
        finally:
            # Restaurar el método original
            search_cv._run_search = original_run_search

print("Iniciando entrenamiento optimizado...")
random_search = fit_with_progress(random_search, X, y)
elapsed = time.time() - start_time

print(f"\nENTRENAMIENTO COMPLETADO!")
print(f"Hora de finalización: {time.strftime('%H:%M:%S')}")
print(f"Tiempo real: {elapsed/60:.2f} minutos")
print(f"Aceleración: {(80*5)/(20*3):.1f}x más rápido que configuración original")

print(f"\nMONITOREO POST-ENTRENAMIENTO:")
final_memory = monitor_memory()

print(f"\n{'='*60}")
print("RESULTADOS DE LA BÚSQUEDA OPTIMIZADA")
print(f"{'='*60}")
print(f"Mejores parámetros: {random_search.best_params_}")
print(f"Mejor score de validación cruzada: {random_search.best_score_:.4f}")
print(f"Modelos evaluados: {len(random_search.cv_results_['mean_test_score'])}")
print(f"Uso final de RAM: {final_memory:.1f}%")

# 6. Análisis del modelo seleccionado
best_model = random_search.best_estimator_
print(f"\nModelo seleccionado:")
print(f"- Kernel: {random_search.best_params_['svc__kernel']}")
print(f"- C: {random_search.best_params_['svc__C']}")
if 'svc__gamma' in random_search.best_params_:
    print(f"- Gamma: {random_search.best_params_['svc__gamma']}")
print(f"- Class weight: {random_search.best_params_['svc__class_weight']}")

# 7. Predicciones en el conjunto de test
print("\nGenerando predicciones para el conjunto de test...")
y_pred_test = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# 8. Validación cruzada detallada
print("\nCalculando métricas de validación cruzada...")
y_pred_cv = cross_val_predict(best_model, X, y, cv=cv)
accuracy_cv = accuracy_score(y, y_pred_cv)

print(f"Accuracy en validación cruzada: {accuracy_cv:.4f}")

# 9. Visualizaciones
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Matriz de confusión
cm = confusion_matrix(y, y_pred_cv)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
axes[0].set_xlabel('Predicho')
axes[0].set_ylabel('Real')
axes[0].set_title(f'Matriz de Confusión\n(Accuracy: {accuracy_cv:.4f})')

# Distribución de predicciones en test
pred_counts = pd.Series(y_pred_test).value_counts().sort_index()
axes[1].bar(pred_counts.index, pred_counts.values, alpha=0.7)
axes[1].set_xlabel('Rendimiento Global')
axes[1].set_ylabel('Número de predicciones')
axes[1].set_title('Distribución de predicciones en Test')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 10. Reporte detallado
print(f"\n{'='*50}")
print("REPORTE DE CLASIFICACIÓN")
print(f"{'='*50}")
print(classification_report(y, y_pred_cv))

# 11. Guardar resultados
submission = pd.DataFrame({
    'ID': test['ID'], 
    'RENDIMIENTO_GLOBAL': y_pred_test
})
submission.to_csv('Resultados_Modelos/SVM/submission_svm_rapido.csv', index=False)

# Guardar también las probabilidades
prob_df = pd.DataFrame(y_pred_proba, columns=best_model.classes_)
prob_df['ID'] = test['ID']
prob_df['RENDIMIENTO_GLOBAL'] = y_pred_test
prob_df.to_csv('Resultados_Modelos/SVM/submission_svm_rapido_probabilidades.csv', index=False)

# Guardar el modelo entrenado
joblib.dump(best_model, 'Resultados_Modelos/SVM/modelo_svm_entrenado.joblib')

print(f"\nArchivos generados:")
print(f"- Resultados_Modelos/SVM/submission_svm_rapido.csv (predicciones)")
print(f"- Resultados_Modelos/SVM/submission_svm_rapido_probabilidades.csv (con probabilidades)")
print(f"- Resultados_Modelos/SVM/modelo_svm_entrenado.joblib (modelo entrenado)")

# 12. Estadísticas adicionales
print(f"\n{'='*50}")
print("ESTADÍSTICAS ADICIONALES")
print(f"{'='*50}")
print(f"Número de vectores de soporte: {best_model.named_steps['svc'].n_support_}")
print(f"Distribución original:")
for clase in sorted(y.unique()):
    count = (y == clase).sum()
    pct = count / len(y) * 100
    print(f"  {clase}: {count} ({pct:.1f}%)")

print(f"\nDistribución predicha en test:")
for clase in sorted(pred_counts.index):
    count = pred_counts[clase]
    pct = count / len(y_pred_test) * 100
    print(f"  {clase}: {count} ({pct:.1f}%)")

CONFIGURACIÓN OPTIMIZADA PARA VELOCIDAD
Cores disponibles: 16
MEMORIA RAM:
  Total: 31.1 GB
  Usada: 6.5 GB (22.8%)
  Disponible: 24.0 GB
RAM con suficiente espacio libre
Configuración RÁPIDA - Menos iteraciones pero más eficiente
Número de características: 19
Número de muestras de entrenamiento: 578824
Número de muestras de test: 296786

BÚSQUEDA RÁPIDA DE HIPERPARÁMETROS
CONFIGURACIÓN OPTIMIZADA PARA VELOCIDAD
CONFIGURACIÓN RÁPIDA:
   • Cores utilizados: 12 de 16 disponibles
   • Iteraciones de búsqueda: 20 (reducido)
   • Validación cruzada: 3-fold (reducido)
   • Total de entrenamientos: ~60 modelos SVM
   • Tiempo estimado: 4.0 minutos

Hora de inicio: 02:27:00
Modo RÁPIDO activado - Menos iteraciones, más velocidad
Iniciando entrenamiento optimizado...
Entrenando 20 modelos SVM...
Número de características: 19
Número de muestras de entrenamiento: 578824
Número de muestras de test: 296786

BÚSQUEDA RÁPIDA DE HIPERPARÁMETROS
CONFIGURACIÓN OPTIMIZADA PARA VELOCIDAD
CONFIGURACIÓN RÁP

Progreso SVM:   0%|          | 0/20 [00:00<?, ?modelo/s]

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Cons

[CV] END svc__C=1, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=41.3min
[CV] END svc__C=1, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=45.5min
[CV] END svc__C=1, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=45.5min


...

[CV] END svc__C=1, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; total time=56.5min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=55.1min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=55.1min


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Cons

[CV] END svc__C=1, svc__class_weight=None, svc__kernel=linear; total time=38.9min
[CV] END svc__C=1, svc__class_weight=None, svc__kernel=linear; total time=40.7min
[CV] END svc__C=1, svc__class_weight=None, svc__kernel=linear; total time=40.7min
[CV] END svc__C=1, svc__class_weight=None, svc__kernel=linear; total time=33.6min
[CV] END svc__C=1, svc__class_weight=None, svc__kernel=linear; total time=33.6min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=54.4min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=54.4min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=57.2min
[CV] END svc__C=100, svc__class_weight=balanced, svc__gamma=0.1, svc__kernel=rbf; total time=57.2min


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2000).  Cons

KeyboardInterrupt: 